<a href="https://colab.research.google.com/github/zy0266/esilv-mlops-crashcourse-24/blob/master/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## configure the env and load the dataset

In [ ]:
#!pip install huggingface-hub

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 549.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 946.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from huggingface_hub import login
print("Hugging Face Hub is working")


from datasets import load_dataset

ds = load_dataset("stanfordnlp/imdb")

Hugging Face Hub is working


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
import pandas

import torch



## check the structure and context of the dataset
1)the dataset type is datasetdict that holds multiple datasets.In this case,including train test unsupervised.every dataset include 2 column.one is text
the other is label .in label column 0 reprensent negative and 1 represent positive.

In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
print(ds["test"]["text"][0])

I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they have to alway

## convert it into Pandas dataframe

In [ ]:
train_df = ds['train'].to_pandas()
test_df = ds['test'].to_pandas()
unsupervised_df = ds['unsupervised'].to_pandas()

In [ ]:
train_df.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
# Make sure to download the punkt tokenizer models if you haven't already
nltk.download('punkt_tab')



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## split it by sentence

In [ ]:
train_ds = ds['train']

In [ ]:
def split_sentences(examples):
    # Tokenize each text into sentences
    sentences = [sent_tokenize(text) for text in examples['text']]

    # Flatten the nested lists and keep track of the label
    flat_sentences = []
    flat_labels = []
    for sentence_list, label in zip(sentences, examples['label']):
        flat_sentences.extend(sentence_list)
        flat_labels.extend([label] * len(sentence_list))  # Duplicate label for each sentence

    return {'sentence': flat_sentences, 'label': flat_labels}

# Apply the function to all splits in the DatasetDict
train_p = train_ds.map(
    split_sentences,
    batched=True,  # Process in batches for efficiency
    remove_columns=['text'],  # Drop the original 'text' column
)


In [ ]:
print(train_p)

Dataset({
    features: ['label', 'sentence'],
    num_rows: 271057
})


In [ ]:
def split_sentences(examples):
    # Tokenize the text into sentences
    sentences = sent_tokenize(examples['text'])
    # Return as a list of dictionaries, one for each sentence
    return {'sentence': sentences}

# Create a Hugging Face dataset
data = {'text': [
    "John and Mary went to the park. It was a sunny day.",
    "Dr. Smith visited the hospital. Jane was with him.",
    "Barack Obama was the 44th president of the United States."
]}
dataset = Dataset.from_dict(data)

# Apply the function to split sentences
split_dataset = dataset.map(
    split_sentences,
    batched=True,  # Ensures multiple rows are processed together for efficiency
    remove_columns=['text'],  # Drop the original 'text' column
    batch_size=32  # Adjust based on your resources
)

# Flatten the results to have one sentence per row
split_dataset = split_dataset.flatten()

In [ ]:
def split_sentences(row):
    sentences = sent_tokenize(row['text'])  # Tokenize the text into sentences
    return pd.DataFrame({'sentence': sentences})

# Apply the function to each row of the DataFrame and reset the index
df_split = pd.concat([split_sentences(row) for _, row in train_df.iterrows()], ignore_index=True)

# Display the new DataFrame
print(df_split)

                                                 sentence
0       I rented I AM CURIOUS-YELLOW from my video sto...
1       I also heard that at first it was seized by U....
2       In particular she wants to focus her attention...
3       In between asking politicians and ordinary den...
4       Really, the sex and nudity scenes are few and ...
...                                                   ...
271052                           Time to end this review.
271053  I have to go the dunny to shake hands with the...
271054  The story centers around Barry McKenzie who mu...
271055  Being about the grossest Aussie shearer ever t...
271056  The songs of Barry McKenzie(Barry Crocker) are...

[271057 rows x 1 columns]


## Name column extraction

## solution
extract first name and last name separately
duplicate rows when it have multiple names


In [ ]:
import tensorflow as tf


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

Num GPUs Available:  1


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer,batch_size=64)



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def extract_persons_for_columns(sentences):
    b_per_list = []
    i_per_list = []

    for sentence in sentences:
        ner_results = nlp(sentence)  # Apply NER to the sentence

        b_per = []
        i_per = []

        # Iterate through the results (list of dictionaries)
        for entity in ner_results:
            if entity['entity'] == 'B-PER' or entity['entity'] == 'I-PER':
                # Append the word, handling subword tokenization
                if entity['word'].startswith("##"):
                    # Append subword to the last token in the list
                    if entity['entity'] == 'B-PER' and b_per:
                        b_per[-1] += entity['word'][2:]  # Remove '##' and merge
                    elif entity['entity'] == 'I-PER' and i_per:
                        i_per[-1] += entity['word'][2:]
                else:
                    # Add a new word to the list
                    if entity['entity'] == 'B-PER':
                        b_per.append(entity['word'])
                    elif entity['entity'] == 'I-PER':
                        i_per.append(entity['word'])

        # Add the result to lists
        b_per_list.append(' '.join(b_per) if b_per else None)
        i_per_list.append(' '.join(i_per) if i_per else None)

    return {'b_per': b_per_list, 'i_per': i_per_list}

# Applying the function on the dataset
train_p = train_p.map(
    lambda batch: extract_persons_for_columns(batch['sentence']),
    batched=True,  # Process multiple sentences in a batch
)


Map:   0%|          | 0/271057 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
print(train_p)

Dataset({
    features: ['label', 'sentence', 'b_per', 'i_per'],
    num_rows: 271057
})


In [ ]:
train_p = train_p.to_pandas()

# Check the resulting DataFrame
print(train_p.head())



   label                                           sentence b_per i_per
0      0  I rented I AM CURIOUS-YELLOW from my video sto...  None  None
1      0  I also heard that at first it was seized by U....  Lena  None
2      0  In particular she wants to focus her attention...  None  None
3      0  In between asking politicians and ordinary den...  None  None
4      0  Really, the sex and nudity scenes are few and ...  None  None


In [ ]:
# Option 1: Using dropna with 'how="all"' to drop rows where both are NaN
train_p = train_p.dropna(subset=['b_per', 'i_per'], how='all')

In [ ]:
def pair_and_expand(df):
    expanded_rows = []
    for _, row in df.iterrows():
        # Handle None values by converting them to empty lists
        b_per_list = row['b_per'] if row['b_per'] is not None else []
        i_per_list = row['i_per'] if row['i_per'] is not None else []

        # Align the lists to avoid Cartesian product duplication
        max_len = max(len(b_per_list), len(i_per_list))

        # Extend lists to match lengths, using empty strings as placeholders
        b_per_list = b_per_list + [''] * (max_len - len(b_per_list))
        i_per_list = i_per_list + [''] * (max_len - len(i_per_list))

        # Append aligned rows to the expanded list
        for b, i in zip(b_per_list, i_per_list):
            expanded_rows.append({
                'sentence': row['sentence'],
                'b_per': b,
                'i_per': i
            })

    # Create a new DataFrame from the expanded rows
    return pd.DataFrame(expanded_rows)

# Convert the lists back into aligned rows
# Ensure b_per and i_per are strings and handle nulls
train_p['b_per'] = train_p['b_per'].fillna('').astype(str).str.split()
train_p['i_per'] = train_p['i_per'].fillna('').astype(str).str.split()



# Pair and expand rows
train_p = pair_and_expand(train_p)


In [ ]:
train_p.head()

,sentence,b_per,i_per
0,I also heard that at first it was seized by U....,Lena,
1,"Even Ingmar Bergman, arguably their answer to ...",Ingmar,Bergman
2,"Even Ingmar Bergman, arguably their answer to ...",John,Ford
3,And those pretentious indie movies like The Br...,Vincent,Gallo
4,And those pretentious indie movies like The Br...,Chloe,Sevigny


In [ ]:
df_split=train_p

In [ ]:
df_split.count()

,0
sentence,160908
b_per,160908
i_per,160908


In [ ]:
(df_split['b_per']=='').sum()

6266

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_split.to_csv('data_p.csv')

In [ ]:
from google.colab import files
files.download('data_p.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## sentiment analyse

In [ ]:
#normalize
import re

def clean_text(text):
    text = re.sub(r'#', '', text)  # Remove the # symbol
    text = re.sub(r'https?:\/\/\S+', '', text)  # Remove hyperlinks
    text = text.replace("  ", " ")  # Remove extra spaces
    text = " ".join([x for x in text.split(" ") if not x.isdigit()])  # Remove numbers
    text = text.lower()  # Convert text to lowercase
    return text
df_split['sentence_normalized'] = df_split['sentence'].apply(lambda x: clean_text(x))

df_split.head(10)

In [ ]:
#tokenize

In [ ]:
#keywords

In [ ]:
#score

In [ ]:
#evaluate

## group by name and rank

In [ ]:
#